In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview

In [2]:
#!import ../Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
string mydeploymentName = apiConfig.deployTurbo;
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeploymentName + " & " + myendpoint + " & " + myAPIKey);

In [19]:
#!import ../plugins/MathPlugin/MathPlugin.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(mydeploymentName, myendpoint, myAPIKey);

var kernel = builder.Build();

In [20]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

In [21]:
// Add the math plugin
//var mathPlugin = kernel.ImportSkill(new MathPlugin(), "MathPlugin");

builder.Plugins.AddFromType<MathPlugin>();

//kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
//ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));

In [26]:
#pragma warning disable SKEXP0060

//var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";

//var ask = "I have $2130.23. How much would I have after it grew by 24% and after I spent $5 on a latte?";
var ask = "If my investment of 2130.23 dollars increased by 23%, how much would I have after I spent $5 on a latte?";
//var ask = "If I have 1000.00 dollars and spend 500.00 dollars, how much is left?";

var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 1: Set the initial investment --}}
{{set "investment" 2130.23}}

{{!-- Step 2: Calculate the increased amount --}}
{{set "increasePercentage" 23}}
{{set "increaseAmount" (Multiply investment (Divide increasePercentage 100))}}
{{set "increasedInvestment" (Add investment increaseAmount)}}

{{!-- Step 3: Calculate the final amount after spending $5 --}}
{{set "spentAmount" 5}}
{{set "finalAmount" (Subtract increasedInvestment spentAmount)}}

{{!-- Step 4: Print the final amount --}}
{{json finalAmount}}


In [27]:

#!import ../Settings/Utils.cs
#pragma warning disable SKEXP0060

var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.ToString(), 100));

Original Plan results:

2615.1829

